# 🍿RNN, GRU LSTM  using Pytorch🍿 

Hi, welcome to my second pytorch series for nlp. Link for the first notebook is [here](https://www.kaggle.com/rushinaik/mission-torch-1).

## WAIT!!!............. IF YOU LIKE MY WORD PLEASE DO UPVOTE 


### **RNN** : 
Recurrent Neural Network. It's a type of neural network which accepts sequence of the data and it works like a human brain.

But how rnn works : So RNN takes sequence of the vectors as a input.The RNN processes the sequence of vectors one by one.While processing, it passes the previous hidden state to the next step of the sequence. The hidden state acts as the neural networks memory which helps to hold the previous vectors information in the hidden state. and in that way RNN tracks the previous information also.



<img src="https://miro.medium.com/max/960/1*TqcA9EIUF-DGGTBhIx_qbQ.gif" width=1000>

# Architecture of RNN : 
It has :
1. Input layer : Vectors which we will feed in RNN layer 
2. previous hidden state : Previous hidden state is the output of previous RNN which are supposed to feed in second layer. It works like we will take input and then we will process the data. we will feed the processed data to next layer in that way the information in sequence vector will be used 
3. Tanh activation : The tanh activation is used to help regulate the values flowing through the network. Tanh function transform the vectors values between -1 to 1. so if we have multplication operation in our NN it will be huge after certain iterations, so here tanh will come handy and convert it in to -1 to 1
4. Concatenation : General Addition of two vectors

![Architecture](https://miro.medium.com/max/1900/1*KxrxyB10ZbOc3xjDneQdhA.gif)

# 🤨 🧐  But Why RNN failed and LSTM & GRU invented?🤨 🧐

### Vanishing Gradient: 
- so as we start to feeding the long sentences to the RNN model the 1st hidden layers sends the information to 2nd hidden layer and 2nd to 3rd hidden layer and this goes on.
- So while backpropagation we adjust the weights but as coming till first layer the gradient descent value becomes so small that it will not do any learning and the information will get lost
- And this was the main reason of born of LSTM & GRU: GATED RECURRENT NEURAL NETWORK Which has update gate, forget gate etc that will look next
![VANISHING](https://miro.medium.com/max/700/1*PYiQa_bNzM8ugYz_D1yvgw.png)


# LSTM and GRU 


<img src ="https://miro.medium.com/max/700/1*yBXV9o5q7L_CvY7quJt3WQ.png" width = 1000/> 

### **LSTM** : 
 
 
LSTM’s are the cell state, and it’s various gates. The cell state act as a transport highway that transfers relative information all the way down the sequence chain. You can think of it as the “memory” of the network. The cell state, in theory, can carry relevant information throughout the processing of the sequence. So even information from the earlier time steps can make it’s way to later time steps, reducing the effects of short-term memory. As the cell state goes on its journey, information get’s added or removed to the cell state via gates. The gates are different neural networks that decide which information is allowed on the cell state. The gates can learn what information is relevant to keep or forget during training.

- will update the theory in details 

## **GRU** 

 - The GRU is the newer generation of Recurrent Neural networks and is pretty similar to an LSTM. GRU’s got rid of the cell state and used the hidden state to transfer information. It also only has two gates, a reset gate and update gate.
 - Update Gate
            The update gate acts similar to the forget and input gate of an LSTM. It decides what information to throw away and what new information to add.
            
- Reset Gate
            The reset gate is another gate is used to decide how much past information to forget.

And that’s a GRU. GRU’s has fewer tensor operations; therefore, they are a little speedier to train then LSTM’s. There isn’t a clear winner which one is better. Researchers and engineers usually try both to determine which one works better for their use case


# PyTorch Implementation

In [ ]:
# importing required libraries 

import pandas as pd

# for pytorch imports
import torch

# for functional dependencies like activation function 
import torch.nn.functional as F

# nn is basic module in Torch which provide different neural network architecture
import torch.nn as nn

# for optimizer
import torch.optim as optim

# CountVectorizer for Bagof words model
from sklearn.feature_extraction.text import CountVectorizer

# for padding .. since the LSTM takes input as sequence so it is said that 
#if we have fixed input string computation will be faster and it will improve performance 
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook

In [ ]:
# GPU ..... vrooom vrooom vroooooooooom !!!!
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Class Sequences :
- it will take text dataset as input and processed the text, tokenize it to sequences, pad it
- function __getitem__ willl return the item at particular index
- __len__ return lenght of the sequence

In [ ]:
class Sequences(Dataset):
    def __init__(self, path, max_seq_len):
        self.max_seq_len = max_seq_len
        df = path
        
        # BOW 
        vectorizer = CountVectorizer(stop_words='english', min_df=0.015)
        vectorizer.fit(df.review.tolist())
        
        # Creating Vocabulary
        self.token2idx = vectorizer.vocabulary_
        
        self.token2idx['<PAD>'] = max(self.token2idx.values()) + 1

        tokenizer = vectorizer.build_analyzer()
        self.encode = lambda x: [self.token2idx[token] for token in tokenizer(x)
                                 if token in self.token2idx]
        self.pad = lambda x: x + (max_seq_len - len(x)) * [self.token2idx['<PAD>']]
        
        sequences = [self.encode(sequence)[:max_seq_len] for sequence in df.review.tolist()]
        sequences, self.labels = zip(*[(sequence, label) for sequence, label
                                    in zip(sequences, df.label.tolist()) if sequence])
        self.sequences = [self.pad(sequence) for sequence in sequences]

    def __getitem__(self, i):
        assert len(self.sequences[i]) == self.max_seq_len
        return self.sequences[i], self.labels[i]
    
    def __len__(self):
        return len(self.sequences)

In [ ]:
data  = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
data['label'] = data['sentiment']
del data['sentiment']
data.head()

__Encoding positive as 1 and negative as 0__

In [ ]:
labeling = {
    'positive':1, 
    'negative':0
}

In [ ]:
data['label'] = data['label'].apply(lambda x : labeling[x])


In [ ]:
data.shape

In [ ]:
# feeding data in class and getting its instance in return 
dataset = Sequences(data, max_seq_len=128)


In [ ]:
len(dataset.token2idx)


In [ ]:
def collate(batch):
    inputs = torch.LongTensor([item[0] for item in batch])
    target = torch.FloatTensor([item[1] for item in batch])
    return inputs, target

batch_size = 2048
train_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate)

In [ ]:


class RNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        batch_size,
        embedding_dimension=100,
        hidden_size=128, 
        n_layers=1,
        device='cpu'
    ):
        super(RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.device = device
        self.batch_size = batch_size
        self.encoder = nn.Embedding(vocab_size, embedding_dimension)
        self.rnn = nn.LSTM(
            embedding_dimension,
            hidden_size,
            num_layers=n_layers,
            batch_first=True,
        )
        self.decoder = nn.Linear(hidden_size, 1)
        
    def init_hidden(self ):
        
        return (torch.randn(self.n_layers, self.batch_size, self.hidden_size).to(self.device),
                torch.randn(self.n_layers, self.batch_size, self.hidden_size).to(self.device) )
       
    
    def forward(self, inputs):
        # Avoid breaking if the last batch has a different size
        batch_size = inputs.size(0)
        if batch_size != self.batch_size:
            self.batch_size = batch_size
            
        encoded = self.encoder(inputs)
        output, hidden = self.rnn(encoded, self.init_hidden())
        #o
        output = self.decoder(output[:, :, -1]).squeeze()
        return output

In [ ]:
model = RNN(
    hidden_size=128,
    vocab_size=len(dataset.token2idx),
    device=device,
    batch_size=batch_size
)
model = model.to(device)
model

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.001)

In [ ]:
model.train()
train_losses = []
for epoch in range(5):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, target in progress_bar:
        inputs, target = inputs.to(device), target.to(device)
        model.zero_grad()
        
        output = model(inputs)
    
        loss = criterion(output, target)
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model.parameters(), 3)

        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)

    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')

In [ ]:
def predict_sentiment(text):
    model.eval()
    with torch.no_grad():
        test_vector = torch.LongTensor([dataset.pad(dataset.encode(text))]).to(device)
        
        output = model(test_vector)
        prediction = torch.sigmoid(output).item()

        if prediction > 0.5:
            print(f'{prediction:0.3}: Positive sentiment')
        else:
            print(f'{prediction:0.3}: Negative sentiment')

In [ ]:
text= "that's nice"
predict_sentiment(text)


In [ ]:
text= "that's worst"
predict_sentiment(text)

In [ ]:
# QUOTE FROM RICH DAD POOR DAD
text= "In school we learn that mistakes are bad, and we are punished for making them. Yet, if you look at the way humans are designed to learn, we learn by making mistakes. We learn to walk by falling down. If we never fell down, we would never walk"
print(text)

predict_sentiment(text)

In [ ]:
text = """I love this car.
This view is amazing.
I feel great this morning.
I am so excited about the concert.
He is my best friend
"""
predict_sentiment(text)

In [ ]:
text="""
I do not like this car.
This view is horrible.
I feel tired this morning.
I am not looking forward to the concert.
He is my enemy
"""
predict_sentiment(text)

References : 

https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

https://towardsdatascience.com/the-exploding-and-vanishing-gradients-problem-in-time-series-6b87d558d22

https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

#### DO UPVOTE ⬆️⬆️⬆️⬆️⬆️⬆️
#### DO COMMENT 💬💬💬💬💬💬💬💬
#### Feel free to post for suggestions 💬💬💬💬💬💬💬💬
<img src ="https://i.pinimg.com/originals/f5/f1/26/f5f12634f6378186aa4f88455b122eda.gif" width=1000 height=800>